In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
!!curl -O http://www.manythings.org/anki/fra-eng.zip


['  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current',
 '                                 Dload  Upload   Total   Spent    Left  Speed',
 '',
 '  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0',
 '  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0',
 '  1 6562k    1  100k    0     0  73341      0  0:01:31  0:00:01  0:01:30 73349',
 ' 15 6562k   15 1017k    0     0   427k      0  0:00:15  0:00:02  0:00:13  427k',
 ' 35 6562k   35 2313k    0     0   684k      0  0:00:09  0:00:03  0:00:06  684k',
 ' 55 6562k   55 3619k    0     0   826k      0  0:00:07  0:00:04  0:00:03  826k',
 ' 75 6562k   75 4922k    0     0   914k      0  0:00:07  0:00:05  0:00:02 1010k',
 ' 94 6562k   94 6227k    0     0   975k      0  0:00:06  0:00:06 --:--:-- 1230k',
 '100 6562k  100 6562k    0     0   988k      0  0:00:06  0:00:06 --:--:-- 1302k']

In [3]:
!tar -xf fra-eng.zip

In [4]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "fra.txt"

In [5]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [6]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [7]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")

Epoch 1/100
125/125 [==============================] - 28s 209ms/step - loss: 1.1405 - accuracy: 0.7365 - val_loss: 1.0309 - val_accuracy: 0.7325
Epoch 2/100
125/125 [==============================] - 26s 208ms/step - loss: 0.8249 - accuracy: 0.7792 - val_loss: 0.8215 - val_accuracy: 0.7755
Epoch 3/100
125/125 [==============================] - 26s 209ms/step - loss: 0.6536 - accuracy: 0.8175 - val_loss: 0.6970 - val_accuracy: 0.7975
Epoch 4/100
125/125 [==============================] - 26s 208ms/step - loss: 0.5701 - accuracy: 0.8352 - val_loss: 0.6279 - val_accuracy: 0.8154
Epoch 5/100
125/125 [==============================] - 23s 187ms/step - loss: 0.5214 - accuracy: 0.8478 - val_loss: 0.6017 - val_accuracy: 0.8227
Epoch 6/100
125/125 [==============================] - 24s 192ms/step - loss: 0.4884 - accuracy: 0.8564 - val_loss: 0.5569 - val_accuracy: 0.8367
Epoch 7/100
125/125 [==============================] - 23s 188ms/step - loss: 0.4616 - accuracy: 0.8632 - val_loss: 0.5333 -

INFO:tensorflow:Assets written to: s2s\assets


INFO:tensorflow:Assets written to: s2s\assets
